In [3]:
from datetime import datetime
from zipline.api import order, symbol, record, order_target
from zipline.algorithm import TradingAlgorithm
from zipline.utils.factory import create_trade_source

ImportError: cannot import name 'load_bars_from_yahoo' from 'zipline.utils.factory' (/usr/local/lib/python3.9/site-packages/zipline/utils/factory.py)

In [4]:

def initialize(context):
    context.i = 0
    context.asset = symbol('AAPL')


In [5]:
def handle_data(context, data):
    # Skip first 300 days to get full windows
    context.i += 1
    if context.i < 300:
        return

    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    short_mavg = data.history(context.asset, 'price', bar_count=100, frequency="1d").mean()
    long_mavg = data.history(context.asset, 'price', bar_count=300, frequency="1d").mean()

    # Trading logic
    if short_mavg > long_mavg:
        # order_target orders as many shares as needed to
        # achieve the desired number of shares.
        order_target(context.asset, 100)
    elif short_mavg < long_mavg:
        order_target(context.asset, 0)

    # Save values for later inspection
    record(AAPL=data.current(context.asset, 'price'),
           short_mavg=short_mavg,
           long_mavg=long_mavg)

In [6]:
%load_ext zipline

In [7]:
%%zipline --start 2016-1-1 --end 2018-1-1
from zipline.api import symbol, order, record

def initialize(context):
    pass

def handle_data(context, data):
    order(symbol('AAPL'), 10)
    record(AAPL=data[symbol('AAPL')].price)


ValueError: no data for bundle 'quandl' on or before 2022-02-27 14:42:28.634900+00:00
maybe you need to run: $ zipline ingest -b quandl